In [1]:
import pandas as pd
import pickle
import time
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
# import matplotlib as plt
from copy import deepcopy
from tqdm.notebook import tqdm as pb


from IPython.core.display import display, HTML
pd.set_option("display.max_rows", 8000)
pd.set_option("display.max_columns", 100)
pd.set_option('display.max_colwidth', -1)
display(HTML("<style>.container { width:99% !important; }</style>"))

<ipython-input-1-d5b8eb279d8e>:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
<ipython-input-1-d5b8eb279d8e>:15: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


# Save files

In [2]:
def dump_file(folder_path, file_name, file_to_dump):
    import pickle
    with open(f'{folder_path}/{file_name}.pkl', 'wb') as file:
        pickle.dump(file_to_dump, file)

# Read Saved Dataset

In [20]:
import pickle
with open('IE4483_Mini_Project/df_train.pkl', 'rb') as file:
    df_train = pickle.load(file)
with open('IE4483_Mini_Project/df_test.pkl', 'rb') as file:
    df_test = pickle.load(file)

# The problem at hand

- Objective: given a set of reviews, predict whether it is a positive (1) or negative (0) review
- This is a text classification problem
- this can be seen as a binary classification problem and LogisticRegression can be used

# Training Data Exploration

- 1 means positive, 0 negative

In [3]:
df_train = pd.read_json('IE4483_Mini_Project/train.json')

In [4]:
# here we see that there are more positive reviews than negative reviews, resulting in an unbalanced dataset
# choose a technique that can be use to balance out the dataset: consider SMOTE

df_train['sentiments'].value_counts()

1    6319
0    1082
Name: sentiments, dtype: int64

In [6]:
df_train

,reviews,sentiments
0,I bought this belt for my daughter in-law for ...,1
1,The size was perfect and so was the color. It...,1
2,"Fits and feels good, esp. for doing a swim rac...",1
3,These socks are absolutely the best. I take pi...,1
4,Thank you so much for the speedy delivery they...,1
...,...,...
7396,"I bought these shirts (black, medium) to wear ...",0
7397,"At first, I thought this scarf might not be th...",1
7398,I am very picky when it comes to bras. I want...,1
7399,"This jacket is wind and water resistant, but n...",0


In [9]:
# get in csv format to check data
df_train.to_csv('train.csv', index=False)

# Training Data - Data Cleaning

There are known steps that we can follow to improve the quality of training data for our model

We test for one text first to see validity before applying it to full dataset

# Create the cleaner function

In [4]:
def text_cleaner(text):
    """
    This functions takes in a text review and does the following steps:
    1. Decode extra \ before any unicode character (emojis)
    2. Handle apostrophe before mapping contractions, replace double quotation with single quotation marks
    3. Map contraction to the original form. e.g. I've -> I Have
    4. Parse text with spacy
    5. Filter punctuations, white space, numbers, URL using spacy methods
    6. Lemmatize each spacy token using .lemma_
    7. Remove any special characters using regular expressions
    8. Remove single length spacy tokens
    9. Spell correction dealing with repeated characters
    """
    
    import codecs
    import unidecode
    import re
    import spacy
    nlp = spacy.load("en_core_web_sm")
    
    contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", 
                   "can't've": "cannot have", "'cause": "because", "could've": "could have", 
                   "couldn't": "could not", "couldn't've": "could not have","didn't": "did not", 
                   "doesn't": "does not", "don't": "do not", "hadn't": "had not", 
                   "hadn't've": "had not have", "hasn't": "has not", "haven't": "have not", 
                   "he'd": "he would", "he'd've": "he would have", "he'll": "he will", 
                   "he'll've": "he will have", "he's": "he is", "how'd": "how did", 
                   "how'd'y": "how do you", "how'll": "how will", "how's": "how is", 
                   "I'd": "I would", "I'd've": "I would have", "I'll": "I will", 
                   "I'll've": "I will have","I'm": "I am", "I've": "I have", 
                   "i'd": "i would", "i'd've": "i would have", "i'll": "i will", 
                   "i'll've": "i will have","i'm": "i am", "i've": "i have", 
                   "isn't": "is not", "it'd": "it would", "it'd've": "it would have", 
                   "it'll": "it will", "it'll've": "it will have","it's": "it is", 
                   "let's": "let us", "ma'am": "madam", "mayn't": "may not", 
                   "might've": "might have","mightn't": "might not","mightn't've": "might not have", 
                   "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", 
                   "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", 
                   "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not",
                   "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", 
                   "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", 
                   "she's": "she is", "should've": "should have", "shouldn't": "should not", 
                   "shouldn't've": "should not have", "so've": "so have","so's": "so as", 
                   "this's": "this is",
                   "that'd": "that would", "that'd've": "that would have","that's": "that is", 
                   "there'd": "there would", "there'd've": "there would have","there's": "there is", 
                       "here's": "here is",
                   "they'd": "they would", "they'd've": "they would have", "they'll": "they will", 
                   "they'll've": "they will have", "they're": "they are", "they've": "they have", 
                   "to've": "to have", "wasn't": "was not", "we'd": "we would", 
                   "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", 
                   "we're": "we are", "we've": "we have", "weren't": "were not", 
                   "what'll": "what will", "what'll've": "what will have", "what're": "what are", 
                   "what's": "what is", "what've": "what have", "when's": "when is", 
                   "when've": "when have", "where'd": "where did", "where's": "where is", 
                   "where've": "where have", "who'll": "who will", "who'll've": "who will have", 
                   "who's": "who is", "who've": "who have", "why's": "why is", 
                   "why've": "why have", "will've": "will have", "won't": "will not", 
                   "won't've": "will not have", "would've": "would have", "wouldn't": "would not", 
                   "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would",
                   "y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",
                   "you'd": "you would", "you'd've": "you would have", "you'll": "you will", 
                   "you'll've": "you will have", "you're": "you are", "you've": "you have" } 
    
    # decoding: unicode_escape for the extra "\" before unicode character, then unidecode
    try:
        decoded = unidecode.unidecode(codecs.decode(text, 'unicode_escape'))    
    except:
        decoded = unidecode.unidecode(text)
    
    # handling Apostrophe, people usually use single quote ' or double quotes " for contraction. We remove double quotation contractions so that we can map it to the correct words later on
    apostrophe_handled = re.sub("’", "'", decoded)
    
    # contraction check, check for any contraction form, and replace with its original form
    expanded = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in apostrophe_handled.split(" ")])
    
    # parse words with  spacy
    parsed = nlp(expanded)
    
    # ============= using spacy library, we can filter out puncutation, white space, numbers to improve quality of content in reviews =============
    final_tokens = []
    for t in parsed:
        # over here for each word in the sentence, we skip numbers, punctuation and white space
        if t.is_punct or t.is_space or t.like_num or t.like_url:
            pass
        else:
            if t.lemma_== '-PRON-':
                # we keep the words that are prononuns here since spacy transforms every pronoun to -PRON-
                final_tokens.append(str(t))
            else:
                # remove special characters using regex
                sc_removed = re.sub("[^a-zA-Z]", '', str(t.lemma_))
                # we check here if there exists any string once we subsitute the characters with blanks. If len = 1 it means it contains special characters
                if len(sc_removed) > 1:
                    final_tokens.append(sc_removed)

    # join back the list of words into a string
    joined = ' '.join(final_tokens)

    # we further correct spelling by shortening repetition of words. example "soooo good" will be transformed into "soo good". not a perfect solution but can be worked on
    spell_corrected = re.sub(r'(.)\1+', r'\1\1', joined)
    
    return spell_corrected

In [97]:
# run cleaner function on dataset
reviews_cleaned = []
reviews = list(df_train['reviews'])
count = 0
total = len(df_train['reviews'])

for r in reviews:
    reviews_cleaned.append(text_cleaner(r))
    count+=1
    print(count, "/", total)

1 / 7401
2 / 7401
3 / 7401
4 / 7401
5 / 7401
6 / 7401
7 / 7401
8 / 7401
9 / 7401
10 / 7401
11 / 7401
12 / 7401
13 / 7401
14 / 7401
15 / 7401
16 / 7401
17 / 7401
18 / 7401
19 / 7401
20 / 7401
21 / 7401
22 / 7401
23 / 7401
24 / 7401
25 / 7401
26 / 7401
27 / 7401
28 / 7401
29 / 7401
30 / 7401
31 / 7401
32 / 7401
33 / 7401
34 / 7401
35 / 7401
36 / 7401
37 / 7401
38 / 7401
39 / 7401
40 / 7401
41 / 7401
42 / 7401
43 / 7401
44 / 7401
45 / 7401
46 / 7401
47 / 7401
48 / 7401
49 / 7401
50 / 7401
51 / 7401
52 / 7401
53 / 7401
54 / 7401
55 / 7401
56 / 7401
57 / 7401
58 / 7401
59 / 7401
60 / 7401
61 / 7401
62 / 7401
63 / 7401
64 / 7401
65 / 7401
66 / 7401
67 / 7401
68 / 7401
69 / 7401
70 / 7401
71 / 7401
72 / 7401
73 / 7401
74 / 7401
75 / 7401
76 / 7401
77 / 7401
78 / 7401
79 / 7401
80 / 7401
81 / 7401
82 / 7401
83 / 7401
84 / 7401
85 / 7401
86 / 7401
87 / 7401
88 / 7401
89 / 7401
90 / 7401
91 / 7401
92 / 7401
93 / 7401
94 / 7401
95 / 7401
96 / 7401
97 / 7401
98 / 7401
99 / 7401
100 / 7401
101 / 74

In [102]:
df_train['reviews_cleaned'] = reviews_cleaned

In [33]:
# assigning pos to 1, neg to 0, for easy reference
df_train['sentiments_classified'] = df_train['sentiments'].apply(lambda x: "pos" if x == 1 else "neg")

In [34]:
df_train[0:10]

,reviews,sentiments,reviews_cleaned,sentiments_calssified,sentiments_classified
0,I bought this belt for my daughter in-law for Christmas and she loved it,1,buy this belt for my daughter in law for Christmas and she love it,pos,pos
1,The size was perfect and so was the color. It looked just like it did on the web page:),1,the size be perfect and so be the color it look just like it do on the web page,pos,pos
2,"Fits and feels good, esp. for doing a swim race. Highly recommend this. \n\nBe careful when wearing board shorts with velcro zipper straps over this material. The velcro from the board shorts tend to stick very easily to the material of this swim trunks. Have to be careful when removing the velcro from any shorts or pants. I had this happen and the velcro almost damaged the swim trunks when I was removing it.",1,fit and feel good esp for do swim race highly recommend this be careful when wear board short with velcro zipper strap over this material the velcro from the board short tend to stick very easily to the material of this swim trunk have to be careful when remove the velcro from any short or pant have this happen and the velcro almost damage the swim trunk when be remove it,pos,pos
3,"These socks are absolutely the best. I take pilates classes and if it is hot your feet can slip off the mat. (Not good in the middle of a side plank!)Anyway these socks keep your feet planted and dry. One note; make sure not to order them big, the idea is that they should be snug",1,these sock be absolutely the good take pilate class and if it be hot your foot can slip off the mat not good in the middle of side plankanyway these sock keep your foot plant and dry note make sure not to order they big the idea be that they should be snug,pos,pos
4,Thank you so much for the speedy delivery they came in time for the rehearsal dinner. I loved the red silk neck ties and they really made all the ushers stand out. They made very nice thank you gifts. We were all pleased.\nmarjorie300,1,thank you so much for the speedy delivery they come in time for the rehearsal dinner love the red silk neck tie and they really make all the usher stand out they make very nice thank you gift we be all pleased marjorie,pos,pos
5,"I bought this shoe in black and bronze. I do a lot of running around and this is a very comfortable shoe, true to size and really cute",1,buy this shoe in black and bronze do lot of run around and this be very comfortable shoe true to size and really cute,pos,pos
6,"These shorts are great for all around use. They look great and are comfortable for swimming or just wearing around. I am in college and wear these to class as they are lightweight and still have enough pockets to carry everything I need. they are alos stiffer than a lot of swim trunks, so they don't flop around too bad if your pockets are ful",1,these short be great for all around use they look great and be comfortable for swimming or just wear around be in college and wear these to class as they be lightweight and still have enough pocket to carry everything need they be alo stiff than lot of swim trunk so they do not flop around too bad if your pocket be ful,pos,pos
7,"They look great. I just tried them on and they are already coming apart. It's too bad, 'cause they look great",0,they look great just try they on and they be already come apart it be too bad because they look great,neg,neg
8,"Very comfortable, but the straps keep coming loose.",1,very comfortable but the strap keep come loose,pos,pos
9,"trans. was great. love the shirt!! I know it's a lot for a t-shirt, but I had to have it!!",1,trans be great love the shirt know it be lot for shirt but have to have it,pos,pos


In [46]:
df_train['sentiments_classified'].value_counts()

pos    6319
neg    1082
Name: sentiments_classified, dtype: int64

In [35]:
# save file
dump_file(folder_path='IE4483_Mini_Project', file_name='df_train', file_to_dump=df_train)

# Create Model to run imbalanced data

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score

# create the vectorizer, logistic_regression model
vectorizer = TfidfVectorizer(stop_words=None, max_features=100000, ngram_range=(1,3))
lr = LogisticRegression()

def logistic_regressor(n_splits, X,Y, pipeline, average_method):
    
    # StratifiedKFold, use this as a tool to improve model performance when dealing with imbalance datasets
    kfold = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=777)
    
    # create the lists to store the different required information
    lr_regressors, predictions, accuracy, precision, recall, f1 = [], [], [], [], [], []
    
    for train, test in kfold.split(X, Y):
        lr_fit = pipeline.fit(X[train], Y[train])
        prediction = lr_fit.predict(X[test])
        scores = lr_fit.score(X[test], Y[test])
        
        lr_regressors.append(lr_fit)
        predictions.append(prediction)
        accuracy.append(scores * 100)
        precision.append(precision_score(Y[test], prediction, average=average_method)*100)
        print('              negative    positive')
        print('precision:',precision_score(Y[test], prediction, average=None))
        recall.append(recall_score(Y[test], prediction, average=average_method)*100)
        print('recall:   ',recall_score(Y[test], prediction, average=None))
        f1.append(f1_score(Y[test], prediction, average=average_method)*100)
        print('f1 score: ',f1_score(Y[test], prediction, average=None))
        print('-'*50)

    print("accuracy: %.2f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
    print("precision: %.2f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
    print("recall: %.2f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
    print("f1 score: %.2f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))
    
    return lr_regressors, predictions

In [6]:
# bulding a pipeline using sklearn to be use for modelling
from sklearn.pipeline import Pipeline
original_pipeline = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', lr)
])

We can see that for negative class, he precision is high but the recall is low. If the sentiment is negative, they are really negative. However, it misses a lot of actual negative sentiments. This is mainly due to the fact that the training dataset has a much higher number of positive reviews to negative reviews

In [93]:
# trying to predict with existing model using a toy dataset
test_string_1 = "This shoe is extremely bad!"
test_string_2 = "This shoe is ver good to wear and comfortable"
test_string_3 = "This shoe is not too bad"
test_string_4 = "This shoe is not so comfortable. It rips apart easily and its not simple to wear"
df_test = pd.Series([test_string_1, test_string_2, test_string_3, test_string_4])

lr_regressors, predictions = logistic_regressor(n_splits = 5, X = df_train.reviews_cleaned, Y = df_train.sentiments_classified, pipeline=original_pipeline, average_method='macro')

In [72]:
# the regressor is always giving positive values because of imbalanced dataset
for n, lr in enumerate(lr_regressors):
    print('logistic regressor',n, "predicted values for test dataset:", lr.predict(df_test))

logistic regressor 0 predicted values for test dataset: ['pos' 'pos' 'pos']
logistic regressor 1 predicted values for test dataset: ['pos' 'pos' 'pos']
logistic regressor 2 predicted values for test dataset: ['pos' 'pos' 'pos']
logistic regressor 3 predicted values for test dataset: ['pos' 'pos' 'pos']
logistic regressor 4 predicted values for test dataset: ['pos' 'pos' 'pos']


# Deal with imbalance dataset

## SMOTE
We want to try 3 different kinds of oversampling methods for the negative class
1. SMOTE
2. RandomOverSampler

In [9]:
from imblearn.pipeline import make_pipeline
from imblearn.over_sampling import SMOTE, RandomOverSampler
smote_pipeline = make_pipeline(vectorizer, SMOTE(random_state=777),lr)

In [11]:
smote_pipeline

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_features=100000, ngram_range=(1, 3))),
                ('smote', SMOTE(random_state=777)),
                ('logisticregression', LogisticRegression())])

We can see a much `higher` recall and precision for the negative class, which is what we desire. High precision and recall for positive class is also present

In [12]:
lr_regressors_smote, predictions_smote = logistic_regressor(n_splits = 5, X = df_train.reviews_cleaned, Y = df_train.sentiments_classified, pipeline=smote_pipeline, average_method='macro')

              negative    positive
precision: [0.70612245 0.96440129]
recall:    [0.79723502 0.94303797]
f1 score:  [0.74891775 0.9536    ]
--------------------------------------------------
              negative    positive
precision: [0.69626168 0.94707741]
recall:    [0.68981481 0.94857595]
f1 score:  [0.69302326 0.94782609]
--------------------------------------------------
              negative    positive
precision: [0.67206478 0.9594485 ]
recall:    [0.76851852 0.93591772]
f1 score:  [0.71706263 0.94753704]
--------------------------------------------------
              negative    positive
precision: [0.70539419 0.96287328]
recall:    [0.78703704 0.94382911]
f1 score:  [0.74398249 0.95325609]
--------------------------------------------------
              negative    positive
precision: [0.65714286 0.95465587]
recall:    [0.74193548 0.93349169]
f1 score:  [0.6969697  0.94395516]
--------------------------------------------------
accuracy: 91.41% (+/- 0.63%)
precision: 82.25

In [17]:
test_string_1 = "This shoe is extremely bad!"
test_string_2 = "This shoe is ver good to wear and comfortable"
test_string_3 = "This shoe is not too bad"
test_string_4 = "This shoe is not so comfortable It rips apart easily and its not simple to wear uncomfortable as hell"
df_test = pd.Series([test_string_1, test_string_2, test_string_3, test_string_4])

In [18]:
for n, lr in enumerate(lr_regressors_smote):
    print('logistic regressor',n, "predicted values for test dataset:", lr.predict(df_test))

logistic regressor 0 predicted values for test dataset: ['neg' 'pos' 'neg' 'pos']
logistic regressor 1 predicted values for test dataset: ['neg' 'pos' 'neg' 'pos']
logistic regressor 2 predicted values for test dataset: ['neg' 'pos' 'neg' 'pos']
logistic regressor 3 predicted values for test dataset: ['neg' 'pos' 'neg' 'pos']
logistic regressor 4 predicted values for test dataset: ['neg' 'pos' 'neg' 'pos']


In [22]:
df_test['sentiments_predicted'] = lr_regressors_smote[0].predict(df_test.reviews)

In [24]:
df_test[0:10]

,reviews,reviews_cleaned,sentiments_predicted
0,I bought 2 sleepers. sleeper had holes in the arm pit area and the other sleeper had a whole where the neck trim should of been sewed on. A real waste of my money,buy sleeper sleeper have hole in the arm pit area and the other sleeper have whole where the neck trim should of be sew on real waste of my money,neg
1,"I dare say these are just about the sexiest things I've ever worn. Oh I've had and have G-strings, have some pretty skimpy ones too. But a crotchless G-String, masqurading as a crotchless pantie, what a concept. Try going outside in a short skirt with nothing under it but these. Might as well be walking around naked under that skirt. But then again I've done that. However, wearing these panties slash G-string just seems sexier than naked",dare say these be just about the sexy thing have ever wear oh have have and have string have some pretty skimpy one too but crotchless String masqurade as crotchless pantie what concept try go outside in short skirt with nothing under it but these might as well be walk around naked under that skirt but then again have do that however wear these pantie slash string just seem sexy than naked,pos
2,"everything about the transaction (price, delivery time, quality of item) was great. I wouldn't hesitate to purchase something again from this seller",everything about the transaction price delivery time quality of item be great would not hesitate to purchase something again from this seller,pos
3,"Not bad for just a shirt. Very durable, and matched my teams colors perfectly. Its just a shirt, but it helped my team and I go on to greatness.....",not bad for just shirt very durable and match my team color perfectly its just shirt but it help my team and go on to greatness,pos
4,"These are truly wrinkle free and longer than the average womans botton down, which I love!! Overall, these are fabulous shirts and you can't beat the price",these be truly wrinkle free and long than the average woman botton down which love overall these be fabulous shirt and you can not beat the price,pos
5,I love naughty monkey! I'm so happy with their shoes! They don't hurt my feet,love naughty monkey be so happy with their shoe they do not hurt my foot,pos
6,I fell in love with this boot when I first saw it. It was on another store's website for 159.95. I found them on amazon for 99.95 and just waited it out. About 2 months later the price dropped to 44.95 and the total purchase was less than 50.00 including shipping. The boots were exactly what I expected and I always get complimented on them. Thanks Amazon,fall in love with this boot when see it it be on another store website for find they on amazon for and just wait it out about month later the price drop to and the total purchase be less than include shipping the boot be exactly what expect and always get compliment on they thank Amazon,pos
7,"These shades are a great buy. Fast shipping, great price, and good quality",these shade be great buy fast shipping great price and good quality,pos
8,"There was no picture of this selection, so I guessed it was a ""ringer"" type t-shirt. Instead it was a white shirt with ugly red contrast stitching. I never wore it, but gave it to Goodwill",there be no picture of this selection so guess it be ringer type shirt instead it be white shirt with ugly red contrast stitching never wear it but give it to Goodwill,neg
9,"This leather briefcase was exactly what I was looking for. The leather is smooth and supple and holds up well. It's big enough to accommodate my 13"" MacBook along with an assortment of other papers, iPod, and so forth. I highly recommend it",this leather briefcase be exactly what be look for the leather be smooth and supple and hold up well it be big enough to accommodate my MacBook along with an assortment of other paper iPod and so forth highly recommend it,pos


## RandomOverSampler

In [28]:
ros_pipeline = make_pipeline(vectorizer, RandomOverSampler(random_state=777), lr)

In [29]:
ros_pipeline

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_features=100000, ngram_range=(1, 3))),
                ('randomoversampler', RandomOverSampler(random_state=777)),
                ('logisticregression', LogisticRegression())])

Again there is relatively high precision and recall for positive classes and improved precision and recall for negative classes. The results is comparable to the SMOTE technique

In [30]:
lr_regressors_ros, predictions_ros = logistic_regressor(n_splits = 5, X = df_train.reviews_cleaned, Y = df_train.sentiments_classified, pipeline=ros_pipeline, average_method='macro')

              negative    positive
precision: [0.69387755 0.96197411]
recall:    [0.78341014 0.94066456]
f1 score:  [0.73593074 0.9512    ]
--------------------------------------------------
              negative    positive
precision: [0.68325792 0.94837172]
recall:    [0.69907407 0.94462025]
f1 score:  [0.69107551 0.94649227]
--------------------------------------------------
              negative    positive
precision: [0.6733871  0.96022727]
recall:    [0.77314815 0.93591772]
f1 score:  [0.71982759 0.94791667]
--------------------------------------------------
              negative    positive
precision: [0.70833333 0.96290323]
recall:    [0.78703704 0.94462025]
f1 score:  [0.74561404 0.95367412]
--------------------------------------------------
              negative    positive
precision: [0.66007905 0.9592502 ]
recall:    [0.76958525 0.93190816]
f1 score:  [0.7106383  0.94538153]
--------------------------------------------------
accuracy: 91.37% (+/- 0.52%)
precision: 82.12

# Exploring incorrectly classified data

In [68]:
idx = [63, 195]
df_wrong = df_test[df_test.index.isin(idx)]

In [69]:
df_wrong

,reviews,reviews_cleaned,sentiments_predicted
63,"These are the BEST panties I've ever had! They don't ride up, they feel like they are not even there and they don't cut into your skin! They are so smooth that my jeans don't pull them up either! If you're tired of digging your panties out, even the victoria secret's barely there panties, buy these! They are much cheaper here on Amazon than at Beall's department store. I paid $11.99 for 3 pair at the store.",these be the BEST pantie have ever have they do not ride up they feel like they be not even there and they do not cut into your skin they be so smooth that my jean do not pull they up either if you be tired of dig your pantie out even the victoria secret be barely there pantie buy these they be much cheap here on Amazon than at Beall department store pay for pair at the store,neg
195,"These are the best!!! When they say skinny-they mean it. I cannot find any jeans that will fit this tall skinny child, and not look to baggy.(which she hates) We still needed the adjustable waist(believe it or not) But the difference was the slim cut through the hinnie and hips-she looks like a regular little girl, instead of a skinny girl with pants that are way to big",these be the good when they say skinny they mean it can not find any jean that will fit this tall skinny child and not look to baggywhich she hate we still need the adjustable waistbelieve it or not but the difference be the slim cut through the hinnie and hip she look like regular little girl instead of skinny girl with pant that be way to big,neg


In [70]:
for lr in lr_regressors_smote:
    print(lr.predict(df_wrong.reviews_cleaned))

['neg' 'neg']
['neg' 'neg']
['neg' 'neg']
['neg' 'neg']
['neg' 'neg']


# Submission Dataset

In [40]:
df_test.to_csv('IE4483_Mini_Project/submission.csv', index=False)

In [ ]:
# e) strengths/weakness of model - what are the weaknesses and strengths of logistic regression? link back to cases in test dataset
# f) feature format choice
# g) sentiment classification without labels - unsupervise learning method (use K-Means clustering)
# h) 3 apparoaches to improve noisy data in classification - collect more data, noise filtering, use unsupervise models, use supervise models - decision trees, feature engineering